### 0. 패키지 불러오기

In [2]:
import pandas as pd

from sklearn.datasets import load_iris

from factor_analyzer import FactorAnalyzer

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

C:\Users\HamIG\AppData\Local\Temp\ipykernel_28796\521845342.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 1. 데이터 전처리

#### 1-0. 원본 데이터 확인 및 결측치 대체

In [25]:
df = pd.read_excel("./data/30year_3.xlsx")

df.head()

,시점,합계출산율,국내총생산(명목 원화표시) (십억원),국민총소득(명목 원화표시) (십억원),노동비용총액 (천원),소비자물가지수,일반적으로사람들을신뢰하는정도,우울인식_평균_점,우울감 경험률,모 육아휴직 사용률(%),모 육아휴직자 수,1인당 민간소비(천원),월평균 사교육비(만원),생활물가지수(2020=100),주관적 건강인지율(%),아파트 매매 실거래 평균가격(만원/㎡),연립다세대 매매 실거래 평균가격(만원/㎡),총근로시간 (시간),월임금총액 (천원)
0,1993,1.654,315181.3,314760.4,1501.9,46.831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994,1.656,372493.4,371554.7,1501.9,49.765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995,1.634,436988.8,435384.8,1726.7,51.995,NaN,NaN,NaN,NaN,NaN,5094.0,NaN,51.995,NaN,NaN,NaN,NaN,NaN
3,1996,1.574,490850.9,489129.4,1870.5,54.555,NaN,NaN,NaN,NaN,NaN,5794.0,NaN,54.555,NaN,NaN,NaN,NaN,NaN
4,1997,1.537,542001.8,539090.8,2082.2,56.977,NaN,NaN,NaN,NaN,NaN,5861.0,NaN,56.977,NaN,NaN,NaN,NaN,NaN


In [27]:
total_missing = df.isnull().sum().sum()
total_missing

164

결측치의 개수는 총 164개로, 결측치 대체 방법으로는 SPSS의 수정회귀대체 방법을 활용한다. 

SPSS에서 회귀대체 사용하는 방법 : https://m.blog.naver.com/7461817/222617761587

결측치 대체 방법 설명 사이트 : https://gooopy.tistory.com/44

+ 결측치를 포함하는 변수를 종속변수로, 나머지 변수를 독립변수로 하는 회귀식을 구성하고, 이렇게 추정된 회귀식의 예측값으로 결측 값을 대체하는 방법을 회귀대체라고 부른다.  
+ 단, 회귀대체는 표준오차가 과소추정되어 p-value가 실제보다 작아지고, 신뢰구간이 좁아질 수 있다는 한계점이 존재한다.
+ 따라서, 본 연구에서는 회귀식에 정규분포 변량을 더한 수정회귀대체 방법을 사용하여 표준오차가 과소추정되는 것을 보완한다. 

![image.png](./image/spss8.png)

데이터를 불러올 때, 시점 변수를 양적 변수로 취급했기 때문에 시점 변수는 제외하고, 나머지 변수들을 회귀대체에 활용할 변수로 집어넣는다. 

![image.png](./image/spss9.png)

![image.png](./image/spss11.png)

#### 1-1. 결측치 대체 후의 데이터 불러오기

In [28]:
df = pd.read_excel("./data/30year_regress2.xlsx")

df.head()

,시점,합계출산율,국내총생산(명목 원화표시) (십억원),국민총소득(명목 원화표시) (십억원),노동비용총액 (천원),소비자물가지수,일반적으로사람들을신뢰하는정도,우울인식_평균_점,우울감 경험률,모 육아휴직 사용률(%),모 육아휴직자 수,1인당 민간소비(천원),월평균 사교육비(만원),생활물가지수(2020=100),주관적 건강인지율(%),아파트 매매 실거래 평균가격(만원/㎡),연립다세대 매매 실거래 평균가격(만원/㎡),총근로시간 (시간),월임금총액 (천원)
0,1993,1.654,315181.3,314760.4,1501.9,46.831,3.570,3.94,5.4,47.8,61469,5475,24.6,60.706,30.9,280.4,212.6,182.8,2555
1,1994,1.656,372493.4,371554.7,1501.9,49.765,3.533,4.85,5.0,51.9,54729,6788,25.6,67.299,30.6,260.6,222.4,184.5,2294
2,1995,1.634,436988.8,435384.8,1726.7,51.995,3.560,5.86,6.4,49.5,57142,5094,26.2,51.995,36.8,284.9,180.3,186.5,2276
3,1996,1.574,490850.9,489129.4,1870.5,54.555,3.729,5.84,5.4,43.8,57862,5794,13.1,54.555,29.3,157.5,101.4,170.5,2059
4,1997,1.537,542001.8,539090.8,2082.2,56.977,3.513,6.51,4.7,48.1,62897,5861,22.7,56.977,28.1,277.5,219.5,190.6,2249


#### 1-2. 결측치 확인

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   시점                       30 non-null     int64  
 1   합계출산율                    30 non-null     float64
 2   국내총생산(명목 원화표시) (십억원)     30 non-null     float64
 3   국민총소득(명목 원화표시) (십억원)     30 non-null     float64
 4   노동비용총액 (천원)              30 non-null     float64
 5   소비자물가지수                  30 non-null     float64
 6   일반적으로사람들을신뢰하는정도          30 non-null     float64
 7   우울인식_평균_점                30 non-null     float64
 8   우울감 경험률                  30 non-null     float64
 9   모 육아휴직 사용률(%)            30 non-null     float64
 10  모 육아휴직자 수                30 non-null     int64  
 11  1인당 민간소비(천원)             30 non-null     int64  
 12  월평균 사교육비(만원)             30 non-null     float64
 13  생활물가지수(2020=100)         30 non-null     float64
 14  주관적 건강인지율(%)             30 

In [30]:
total_missing = df.isnull().sum().sum()
total_missing

0

데이터의 결측치를 세어 본 결과, 0개로 모든 결측치가 채워졌다. 

### 1-3. 합계출산율, 시점 제거

In [31]:
# 원본 데이터를 유지하기 위해 데이터프레임을 복사
df_copy = df.copy()

In [32]:
df_copy.drop(["합계출산율", "시점"],axis=1,inplace=True)

In [33]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   국내총생산(명목 원화표시) (십억원)     30 non-null     float64
 1   국민총소득(명목 원화표시) (십억원)     30 non-null     float64
 2   노동비용총액 (천원)              30 non-null     float64
 3   소비자물가지수                  30 non-null     float64
 4   일반적으로사람들을신뢰하는정도          30 non-null     float64
 5   우울인식_평균_점                30 non-null     float64
 6   우울감 경험률                  30 non-null     float64
 7   모 육아휴직 사용률(%)            30 non-null     float64
 8   모 육아휴직자 수                30 non-null     int64  
 9   1인당 민간소비(천원)             30 non-null     int64  
 10  월평균 사교육비(만원)             30 non-null     float64
 11  생활물가지수(2020=100)         30 non-null     float64
 12  주관적 건강인지율(%)             30 non-null     float64
 13  아파트 매매 실거래 평균가격(만원/㎡)    30 non-null     float64
 14  연립다세대 매매 실거래 평균가격(만원/㎡)  30 

### 1-4. 표준화

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_df = scaler.fit_transform(df_copy)

scaled_df = pd.DataFrame(scaled_df, columns=df_copy.columns)

In [35]:
scaled_df.head()

,국내총생산(명목 원화표시) (십억원),국민총소득(명목 원화표시) (십억원),노동비용총액 (천원),소비자물가지수,일반적으로사람들을신뢰하는정도,우울인식_평균_점,우울감 경험률,모 육아휴직 사용률(%),모 육아휴직자 수,1인당 민간소비(천원),월평균 사교육비(만원),생활물가지수(2020=100),주관적 건강인지율(%),아파트 매매 실거래 평균가격(만원/㎡),연립다세대 매매 실거래 평균가격(만원/㎡),총근로시간 (시간),월임금총액 (천원)
0,-1.531379,-1.517218,-1.657200,-1.819357,1.129453,-0.288134,-0.718456,-1.226186,-0.715778,-1.495448,-0.415030,-1.189403,-0.813128,-0.556992,-0.790823,0.631507,-0.341052
1,-1.428478,-1.416657,-1.657200,-1.653117,0.851842,0.274129,-1.222635,-0.639335,-1.712056,-1.196580,-0.218644,-0.792050,-0.872916,-0.817178,-0.656086,0.787699,-0.988744
2,-1.312679,-1.303639,-1.486380,-1.526766,1.054423,0.898178,0.541993,-0.982858,-1.355377,-1.582172,-0.100812,-1.714405,0.362719,-0.497859,-1.234905,0.971455,-1.033412
3,-1.215972,-1.208478,-1.377110,-1.381717,2.322432,0.885821,-0.718456,-1.798725,-1.248949,-1.422837,-2.673473,-1.560117,-1.132001,-2.171988,-2.319675,-0.498590,-1.571915
4,-1.124132,-1.120015,-1.216244,-1.244488,0.701782,1.299794,-1.600770,-1.183246,-0.504697,-1.407586,-0.788164,-1.414146,-1.371156,-0.595100,-0.695958,1.348153,-1.100415


### 2. 요인분석

https://zephyrus1111.tistory.com/462

https://blog.naver.com/wlgid0727/220389419955

#### 적합성 검정 

주어진 데이터가 요인분석하기에 적합한지 확인하는 검정으로, Bartlett 구형성 검정과 KMO Test를 통해 확인한다. 

Bartlett 검정 : https://m.blog.naver.com/pmw9440/221969888263

KMO Test : https://en.wikipedia.org/wiki/Kaiser%E2%80%93Meyer%E2%80%93Olkin_test

Bartlett 구형성 검정

주어진 변수들에 대한 상관 행렬과 항등 행렬을 비교하는 검정법으로, 변수 간의 상관관계가 있는 지를 확인하는 검정 방법이다. 검정통계량으로는 카이제곱 통계량을 사용한다.

귀무가설(H0) : 주어진 데이터의 상관행렬은 단위행렬과 동일하다.\
대립가설(H1) : 주어진 데이터의 상관행렬은 단위행렬과 동일하지 않다.

![image.png](./image/chi.png)

데이터로부터 계산된 상관행렬이 단위행렬과 동일하다는 것은 비대각원소가 0이라는 뜻으로, 변수들 간의 상관계수가 0이라는 의미이다. 이는 변수들 간에 상관관계가 없다는 해석이 가능하다. 

In [36]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

chi_square_value,p_value=calculate_bartlett_sphericity(scaled_df)

chi_square_value, p_value

(1001.7600237445123, 7.128432803470782e-132)

Bartlett 구형성 검정 결과, 카이제곱 통계량은 1001, p-value = 0.000...으로 유의수준 5%에서 귀무가설을 기각한다.\
따라서, 주어진 데이터의 상관행렬이 단위행렬과 동일하지 않기 때문에 변수들 간의 상관관계가 존재한다고 볼 수 있다. 

KMO(Kaiser–Meyer–Olkin) Test

변수들 간의 상관관계가 다른 변수에 의해 잘 설명되는지 측정하는 지표로, 다음과 같이 계산한다. 

![image.png](./image/KMO_value2.png)

![image.png](./image/기준.png)

상관계수와 편상관계수 이해 : https://csshark.tistory.com/189

편상관계수는 다른 변수의 영향을 제거한 상태에서 두 변수 간의 순수한 상관관계를 분석하는 통계적 기법

KMO 값이 0에 가까우면 상관계수의 합에 비해 편상관계수의 합이 크다는 것을 의미한다. 

상관계수가 클수록 두 변수 간의 상관관계를 고려할 때 다른 변수의 영향을 같이 생각해야한다는 뜻이므로, 여러 변수 간의 관계를 동시에 보고 요인을 묶는 요인분석의 목적에 적합하다. 

In [37]:
from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all,kmo_model=calculate_kmo(scaled_df)

kmo_model

c:\Users\HamIG\AppData\Local\Programs\Python\Python311\Lib\site-packages\factor_analyzer\utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


0.7508400148240847


KMO Test 결과, KMO value = 0.75로 주어진 데이터에 대해 요인분석을 시행하기에 적합하다고 판단한다. 

#### 고유값 개수 결정 

주성분 분석을 시행하여 주어진 데이터의 공분산행렬을 구하고, 공분산행렬로부터 고유값을 계산하여 요인분석에 적합한 요인의 개수를 결정한다.  

In [38]:
fa = FactorAnalyzer(method = 'principal', n_factors=10, rotation='varimax').fit(scaled_df)

In [39]:
ev,v=fa.get_eigenvalues()
print('고유값:\n', ev)

고유값:
 [9.45105105e+00 3.23494060e+00 1.54200399e+00 8.90491757e-01
 5.91621341e-01 4.26496311e-01 3.04325818e-01 1.68562905e-01
 1.44493324e-01 1.26973527e-01 5.86689674e-02 2.75066225e-02
 2.46841699e-02 4.31775619e-03 2.30144412e-03 1.55289700e-03
 7.52840459e-06]


총 3개의 고유값이 1보다 크지만, 해당 요인들로 요인분석을 진행하면 요인에 의해 설명되는 총 분산이 80% 미만으로 주어진 데이터를 충분히 설명하기에 부족하다고 판단했다.\
따라서, 본 연구에서는 총 분산의 80%을 가져가는 것을 지향하여 요인 수를 4개로 설정한다. 

먼저, 데이터프레임의 모든 행과 열을 출력하도록 옵션을 설정한다. 

In [40]:
# 데이터프레임 전체 출력 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### 요인분석 객체 생성

FactorAnalyzer 함수를 활용해서 요인분석 객체를 생성하되, method = ml로 지정한다. 

FactorAnalyzer의 method = {ml, mle, minres, principal, uls}
* ml or mle -> Maximum Likelihood (최대가능도함수)
* uls or minres-> Minimum Residual (잔차 제곱합 최소화)
* principal -> Principal Analysis (주성분 분석)

In [41]:
fa = FactorAnalyzer(method = 'ml', n_factors=4, rotation='varimax').fit(scaled_df)

요인적재량 구하기

각 변수와 요인 간의 상관관계 정도를 나타내는 값으로, 각 변수가 특정 요인과 얼마나 관련되어 있는지를 보여주는 지표이다.

In [43]:
# 요인적재값 구해서 데이터프레임에 저장
loadings_df = pd.DataFrame(fa.loadings_).round(4)

# 열 이름 변경 -> 요인1 ~ 요인4
loadings_df.columns = ['Factor1', 'Factor2', 'Factor3', 'Factor4']

# 행 이름 변경 -> df_copy의 열(변수명)
loadings_df.index = df_copy.columns

loadings_df

,Factor1,Factor2,Factor3,Factor4
국내총생산(명목 원화표시) (십억원),0.9315,0.3064,-0.1258,0.1406
국민총소득(명목 원화표시) (십억원),0.9307,0.3078,-0.1291,0.1397
노동비용총액 (천원),0.9400,0.2997,-0.0439,0.1415
소비자물가지수,0.9607,0.2366,-0.0184,0.1224
일반적으로사람들을신뢰하는정도,-0.2654,-0.6741,-0.4312,-0.1388
우울인식_평균_점,0.2928,-0.1086,-0.4737,-0.1324
우울감 경험률,0.1541,0.1152,0.4946,0.5450
모 육아휴직 사용률(%),0.2317,0.7369,0.2591,0.5453
모 육아휴직자 수,0.2753,0.3746,0.0941,0.5966
1인당 민간소비(천원),0.9374,0.2957,-0.0749,0.1384


변수별 요인적재량을 기준으로 해서 요인적재량이 높은 변수들끼리 같은 요인으로 묶는다. 

요인 1 : 국내총생산, 국민총소득, 노동비용총액, 소비자물가지수, 민간소비\
요인 2 : 아파트 가격, 연립다세대 가격, 월임금\
요인 3 : 주관적 건강인지율, 우울감 경험률, 우울인식점수\
요인 4 : 모 육아휴직자 수, 모 육아휴직 사용률

#### 결과 정리

요인별 적재량 제곱합, 요인별 분산, 누적분산

SS Loadings : 요인적재량의 제곱으로, 각 요인이 데이터를 얼마나 잘 설명하는지 나타내는 지표\
Proportion Var : 총 분산 중에서 해당 요인이 차지하는 분산\
Cumulative Var : 요인들의 누적 분산 

In [108]:
# 요인별 누적 분산 구하기 

factor_variance = fa.get_factor_variance()

factor_variance_df = pd.DataFrame(factor_variance, columns=['Factor1', 'Factor2', 'Factor3', 'Factor4'], index=['SS Loadings', 'Proportion Var', 'Cumulative Var'])

factor_variance_df

,Factor1,Factor2,Factor3,Factor4
SS Loadings,7.149523,5.088690,1.564288,1.225100
Proportion Var,0.397196,0.282705,0.086905,0.068061
Cumulative Var,0.397196,0.679901,0.766806,0.834867


요인점수 

N차원의 데이터를 P차원으로 축소시켰을 때의 값 (P < N)\
ex) 3차원 공간에서 (3, 2, 1)의 값을 가지는 데이터를 2차원 공간으로 축소시키면 (0.2, 0.5)인 데이터로 변한다.

요인분석을 통해 도출된 소수의 요인들로 다수의 변수들을 축약할 수 있는데, 이때 각 개인이 그 요인들에 얼마나 해당되는지를 나타낸다.

In [109]:
factor_score_df = pd.DataFrame(fa.transform(scaled_df).round(4))

# 열 이름 변경 
factor_score_df.columns = ['Factor1', 'Factor2', 'Factor3', 'Factor4']

# 요인 점수와 합계출산율 결합
factor_score_df['합계출산율'] = df['합계출산율']

# '합계출산율' 열을 맨 앞으로 가져오기
factor_score_df = factor_score_df[['합계출산율'] + [col for col in factor_score_df.columns if col != '합계출산율']]

factor_score_df

,합계출산율,Factor1,Factor2,Factor3,Factor4
0,1.654,-1.6786,0.0154,-1.1479,-0.8109
1,1.656,-1.5669,-0.0899,-0.9672,-0.3260
2,1.634,-1.4183,-0.5743,-0.8145,0.1273
3,1.574,-1.0444,-1.8488,-1.8127,0.4195
4,1.537,-1.1731,-0.2839,-1.0889,-0.7651
5,1.464,-1.0484,-0.2088,0.1637,-0.3473
6,1.425,-0.9553,-0.0212,0.4182,-0.6181
7,1.480,-1.0645,0.5456,0.4952,0.1119
8,1.309,-1.1712,1.0577,0.4405,0.4227
9,1.178,-0.9350,1.0524,1.2827,0.2133


### 3. 결과 저장 

분산표 

In [282]:
file_path = 'variance_regress.xlsx'
factor_variance_df.to_excel(file_path, index=False)

요인적재량

In [283]:
loadings_df_copy = loadings_df.copy()

loadings_df_copy['Variable'] = loadings_df_copy.index

file_path = 'loadings_regress.xlsx'
loadings_df_copy.to_excel(file_path, index=False)

요인점수

In [284]:
file_path = 'score_regress.xlsx'
factor_score_df.to_excel(file_path, index=False)